In [ ]:
import gymnasium as gym
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from IPython.display import display
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.utils import set_random_seed
from custom_env.redes_opticas_env import RedesOpticasEnv
import time

def make_env(num_ont, v_max_olt=10e6,vt_contratada=10e6/10, n_ciclos=200, rank=0, seed=0):
    def _init():
        env = RedesOpticasEnv(render_mode=None,seed=seed, num_ont=num_ont, v_max_olt=v_max_olt, vt_contratada=vt_contratada,n_ciclos=n_ciclos)
        return env
    return _init

def transpuesta(list_ont):
    np_array = np.array(list_ont)

    transposed_np_array = np_array.T

    list_transpuesta = transposed_np_array.tolist()

    return list_transpuesta

def grafico_pareto(valores_instantes, ont):

    valoresInstantesFinales=[]
    valoresPareto=[]

    # Modificar los valores de Pareto en posiciones pares donde el instante es 0
    for i in valores_instantes:
        cont=0
        for j in i:
            if cont%2==0:
                if j==0:
                    valoresInstantesFinales.append(j)
                    valoresPareto.append(0)
                else:
                    valoresInstantesFinales.append(j)
                    valoresPareto.append(1)
            else:
                if j==0:
                    valoresInstantesFinales.append(j)
                    valoresPareto.append(0)
                else:
                    valoresInstantesFinales.append(j)
                    valoresPareto.append(0)

            cont+=1

    extended_instantes = []
    extended_pareto = []
    
    current_time = 0
    
    # Añadir valores extendidos para mantener el gráfico en el eje y hasta el siguiente instante
    for i, valor in enumerate(valoresInstantesFinales):
        current_time += valor
        # Añadir el tiempo actual y su valor correspondiente de Pareto
        extended_instantes.append(current_time)
        if i < len(valoresPareto) - 1:
            extended_pareto.append(valoresPareto[i])
            extended_instantes.append(current_time)
            extended_pareto.append(valoresPareto[i+1])
        else:
            extended_pareto.append(valoresPareto[i])
    
    plt.figure(figsize=(12, 6))
    plt.step(extended_instantes, extended_pareto, where='post', linestyle='-', color='blue')
    plt.xlabel('Tiempo Acumulado')
    plt.ylabel('Valores (Pareto)')
    plt.title(f'Gráfica de valores de Pareto de la ONT {ont+1}')
    plt.xlim([0,n_ciclos/5])
    plt.grid(True)
    plt.show()

# Funcion auxiliar para el array de los instantes de los ciclos y luego calcular la grafica de barras
def funcion_aux(array_valores):

    x=[[] for i in range(num_ont)]

    for i, subarray in enumerate(array_valores):
        for j, sublist in enumerate(subarray):
            for k, valor in enumerate(sublist):
                x[j].append(valor)

    return x


if __name__ == "__main__":
    env_id = 'RedesOpticasEnv-v0'  # Hay que asegurarse de que este ID coincida con el registrado
    num_test = 20
    seed = np.random.randint(0, 10)
    num_envs = 1  # Número de entornos paralelos
    num_ont=16
    v_max_olt=10e9 # 10 Gpbs (XGSPON) 
    T=0.002
    OLT_Capacity=v_max_olt*T
    vt_contratada=600e6
    Max_bits_ONT=vt_contratada*T

    n_ciclos=int(input("Cuantos ciclos quiere ver: "))

    vec_env = DummyVecEnv([make_env(num_ont, v_max_olt,vt_contratada,n_ciclos, rank=i, seed=42) for i in range(num_envs)])    
    
    n_steps = 16384  # Steps por actualización
    batch_size = 256  # Tamaño del mini-batch (16384 es múltiplo de 256)

    model = PPO(
        "MlpPolicy",
        vec_env,
        verbose=1,
        n_steps=n_steps,
        batch_size=batch_size,
        learning_rate=0.00025,
        gamma=0.99,
        gae_lambda=0.95
    )

    start_time = time.time()

    model.learn(total_timesteps=1000)

    end_time = time.time()

    training_time = end_time - start_time
    print(f"El tiempo de entrenamiento fue de {training_time} segundos.")

    # Fase de pruebas
    num_test_episodes = 1
    episode_info = []  

    # Lista de en cada ont guardar el valor de su capacidad, de entrada salida y del pendiente(entrada-salida)
    list_ont = []
    list_ont_2 = []
    list_pendiente=[]

    estados_on_off_recolectados = []

    # Capacidad de la OLT
    tamano_cola=[]

    obs = vec_env.reset()
    _states = None
    for episode in range(num_test_episodes):
        
        done = np.array([False]*num_envs)
        step_counter = 0

        while step_counter < n_ciclos:

            action, _states = model.predict(obs, state=_states, deterministic=True)
            obs, rewards, dones, info = vec_env.step(action)

            episode_info.append(info)
            for i in range(len(info)):  # Itera sobre cada sub-entorno
                suma = 0
                list_ont.append(info[i]['trafico_entrada'])
                list_ont_2.append(info[i]['trafico_salida'])
                list_pendiente.append(info[i]['trafico_pendiente'])
                estados_on_off_recolectados.append(info[i]['trafico_IN_ON_actual'])
            
            done |= dones  # Actualiza 'done' para todos los entornos
            step_counter += 1


    list_transpuesta=transpuesta(list_ont)
    array_transpuesta = np.array(list_transpuesta) / 2000 # 0.002*10e6(transformacion de bps a Mbps)=2000
    list_valores_entrada = array_transpuesta.tolist()

    list_transpuesta_2=transpuesta(list_ont_2)
    array_transpuesta_2 = np.array(list_transpuesta_2) / 2000 # 0.002*10e6(transformacion de bps a Mbps)=2000
    list_valores_salida = array_transpuesta_2.tolist()

    list_pendiente_transpuesta=transpuesta(list_pendiente)
    array_pendiente_transpuesta = np.array(list_pendiente_transpuesta) / 100000 # 0.002*1e6(transformacion de bps a Mbps)=2000
    list_pendiente_fin = array_pendiente_transpuesta.tolist()

    maximo_pendiente=[]
    for i in range(num_ont):
        maximo_pendiente.append(max(list_pendiente_fin[i]))

    valoresInstantes=funcion_aux(estados_on_off_recolectados) # Hallar valores de los instantes para el grafico de barras

    # Graficas
    for i in range(num_ont):

        nuevo_x = np.arange(2, 2 * n_ciclos + 1, 2)  # x se duplica para cada punto

        # Grafica del trafico de pareto de las redes
        plt.figure(figsize=(12, 6))
        plt.ylim(0,1100)
        plt.xlim(0, 2 * n_ciclos + 1)
        plt.xlabel('Tiempo en milisegundos')
        plt.ylabel('Ancho de banda en Mbps')
        plt.plot(nuevo_x, list_valores_entrada[i], label=f'Tráfico de entrada de la ONT {i+1}')
        plt.plot(nuevo_x, list_valores_salida[i], label=f'Tráfico de salida de la ONT {i+1}')
        plt.title(f'Grafica del trafico de entrada y salida de la ONT {i+1} en Mbps')
        plt.legend()
        plt.show()
        grafico_pareto(valoresInstantes[i], i)

        # Grafica del trafico pendiente de la ont determinada
        plt.figure(figsize=(12, 6))
        plt.ylim(0,maximo_pendiente[i])
        plt.xlim(0, 2 * n_ciclos + 1)
        plt.xlabel('Tiempo en milisegundos')
        plt.ylabel('Tamaño de la cola en Mbits')
        plt.plot(nuevo_x,list_pendiente_fin[i], label=f'Grafica del trafico pendiente de la ONT {i+1}')
        plt.title(f'Grafica del trafico pendiente de la ONT {i+1} en Mbits en el ciclo determinado')
        plt.show()
        
